In [ ]:
%pip install qdrant-client
%pip install openai
%pip install sentence-transformers 

In [ ]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from openai import OpenAI, AzureOpenAI
from qdrant_client.models import Distance, VectorParams, PointStruct
from sentence_transformers import SentenceTransformer

### all-MiniLM-L6-v2

- **Architecture**: A lightweight and efficient version of Microsoft's larger models, based on the Transformer architecture with 6 layers.
- **Purpose**: Designed for natural language understanding and generation, optimized for performance on edge devices.
- **Language Coverage**: Supports multiple languages, suitable for international applications.
- **Usage**: Ideal for scenarios where model size and processing speed are crucial, such as mobile devices or web applications.
- **Performance**: Offers a good balance between performance and efficiency, despite its smaller size.
- **Quality of Embeddings**: Generates high-quality, dense text representations, efficiently capturing semantic meanings.
- **Dimensionality**: Produces embeddings with lower dimensionality, reducing computational requirements while maintaining effectiveness.
- **Similarity and Clustering Tasks**: Useful for semantic similarity measurements, document clustering, and information retrieval systems.

In [ ]:

stmodel = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
azure_api_key = os.environ.get("AZURE_OPENAI_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")

In [ ]:
# Azure OpenAI API
model = "gpt-35-turbo"
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-02-15-preview"
)

In [ ]:
# OpenAI API
model = "gpt-3.5-turbo"
openai_client = OpenAI(api_key=api_key)

In [ ]:
client = QdrantClient(host='localhost', port=6333)

In [ ]:
client.delete_collection(collection_name="hockey")

In [ ]:
client.create_collection(
    collection_name="hockey",
    vectors_config=VectorParams(size=384, distance=Distance.DOT),
)

In [ ]:
documents = [
    "Automatic Game Misconduct – An automatic game misconduct shall be applied to any player who has been assessed a third major penalty, or second fighting major penalty, in the same game. The automatic game misconduct penalty due to the third major penalty, or second fighting major penalty, shall be waived for a player in the altercation if the opposing player was assessed an instigator penalty in either altercation.",
    "Match Penalty – A match penalty involves the suspension of a player for the balance of the game and the offender shall be ordered to the dressing room immediately.",
    "Game Misconduct Penalty – A game misconduct penalty involves th suspension of a player for the balance of the game but a substitute is permitted to replace the player removed. Ten minutes are applied in the league records to the player incurring a game misconduct.",
    "In regular League games, any player who incurs a total of ten (10) fighting major penalties shall be suspended automatically for the next League game of his team. For each subsequent fighting major penalty the player shall be suspended automatically for the next League game of his team.",
    "In regular League games, any player who incurs a total of fourteen (14) fighting major penalties shall then be suspended automatically for the next two (2) League games of his team. For each subsequent fighting major penalty the player shall be suspended automatically for the next two (2) League games of his team." ,
    "In addition, the following list of infractions can also result in a game misconduct penalty being assessed: (i) interfering with or striking a spectator (ii) racial taunts or slur (iii) spitting on or at an opponent or spectator",
    "Boarding - A boarding penalty shall be imposed on any player who checks or pushes a defenseless opponent in such a manner that causes the opponent to hit or impact in the boards violently or dangerously. The everity of the penalty, based upon the impact with the boards, shall be at the discretion of the Referee."
]
metadata = [f"AHL Rulebook" for i in range(len(documents))]
ids = [i for i in range(len(documents))]

In [ ]:
vectors = stmodel.encode(documents) 

In [ ]:
metadata = ["AHL Rulebook" for _ in range(len(documents))]
ids = list(range(len(documents)))

In [ ]:
points = [
    PointStruct(id=id_, vector=vector.tolist(), payload={"text": doc, "metadata": meta})
    for id_, vector, doc, meta in zip(ids, vectors, documents, metadata)
]

In [ ]:
operation_info = client.upsert(
    collection_name="hockey",
    wait=True,
    points=points
)

In [ ]:
query = "What is a game misconduct penalty?"
query_vector = stmodel.encode([query])[0]


In [ ]:
print(query_vector)

In [ ]:
search_result = client.search(
    collection_name="hockey", query_vector=query_vector, limit=3
)

print(search_result)

In [ ]:
results = []
for s in search_result:
    print(s)
    results.append(s.payload["text"])

# turn results into a string
results = "\n".join(results)

In [ ]:
system_message = "You are an expert in hockey rules.  Use the references from the AHL Rulebook to answer the question."
prompt = f"""Use the QUERY below, along with the CONTEXT to answer the query.  Do not use any other sources to answer the question.  If you do not know the answer, answer 'I don't know.

== QUERY ==
{query}
== END QUERY ==

== CONTEXT ==
{results}
== END CONTEXT ==
"""

messages = [
    { "role": "system", "content": system_message },
    { "role": "user", "content": prompt }
]

In [ ]:
print(prompt)

In [ ]:
model = "gpt-3.5-turbo"

response = openai_client.chat.completions.create(
    model=model,
    messages=messages
)

In [ ]:
print(response.choices[0].message.content)